In [105]:
import pandas as pd
from gurobipy import *
import numpy as np

In [106]:
# Reading the data from the excel sheets
source_df = pd.read_excel("Source_facility_info.xlsx", sheet_name="PZA")
destination_df = pd.read_excel("Destination_facility_info.xlsx", sheet_name="PZE")
trucking_df = pd.read_excel("Trucking_info.xlsx", sheet_name="Truck")

# Convert relevant columns to datetime and normalize shift times
def normalize_shift_times(row):
    start_hour = row['Start of shift'].hour + row['Start of shift'].minute / 60
    end_hour = row['End of lay-on'].hour + row['End of lay-on'].minute / 60
    if end_hour < start_hour:
        end_hour += 24  # normalize end_hour for the next day
    return pd.Series([start_hour, end_hour])

destination_df[['Start of shift', 'End of lay-on']] = destination_df.apply(normalize_shift_times, axis=1)
source_df['planned_end_of_loading'] = pd.to_datetime(source_df['planned_end_of_loading'])



In [107]:
# Initialize the model
model = Model("DHL_Optimization")

# Decision Variables
source_list = list(trucking_df['Origin_ID'].unique()[:5])  # Index of PZA
destination_list = list(trucking_df['Destination_ID'].unique()[:1])  # Index of PZE
routes_list = [(i, j) for i in source_list for j in destination_list if i != j]
consignment_list = [
    x for (i, j) in routes_list
    for x in source_df[(source_df['Origin_ID'] == i) & (source_df['Destination_ID'] == j)]['id'].values
]

valid_combinations = [(i, j, k) for (i, j) in routes_list for k in consignment_list if k in source_df[(source_df['Origin_ID'] == i) & (source_df['Destination_ID'] == j)]['id'].values]

trucks = range(300)

# Decision Variables
X = {}
Z = {}
T = {}
# ArrivalDay = {}
ArrivalTime = {}
ArrivalDayBinary = {}

for (i, j, k, l) in [(i, j, k, l) for (i, j, k) in valid_combinations for l in trucks]:
    X[(i, j, k, l)] = model.addVar(vtype=GRB.BINARY, name=f"X_{i}_{j}_{k}_{l}")

for l in trucks:
    Z[l] = model.addVar(vtype=GRB.BINARY, name=f"Z_{l}")
    T[l] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"T_{l}")
    #ArrivalDay[l] = model.addVar(lb = 1, ub=4, vtype=GRB.INTEGER, name=f"ArrivalDay_{l}")
    ArrivalTime[l] = model.addVar(lb=0, ub=24, vtype=GRB.CONTINUOUS, name=f"ArrivalTime_{i}_{j}_{k}_{l}")

    for d in range(1, 7):  # Maximum number of days to consider
        ArrivalDayBinary[(l, d)] = model.addVar(vtype=GRB.BINARY, name=f"ArrivalDayBinary_{l}_{d}")


In [108]:
# Objective function: Minimize the total arrival time
model.setObjective(quicksum(Z[l]* quicksum(d * ArrivalDayBinary[(l, d)] for d in range(1, 7)) for l in trucks) ,GRB.MINIMIZE)

In [109]:
# Constraints
# 1. Each truck can carry at most 2 consignments
for l in trucks:
    model.addConstr(quicksum(X[(i, j, k, l)] for (i, j, k) in valid_combinations) <= 2 * Z[l])

In [110]:
# 2. Consignment can only be released after the latest release time of the consignments
for (i, j, k) in valid_combinations:
    release_time = source_df[source_df['id'] == k]['planned_end_of_loading'].dt.hour.values[0]  # Convert to hours
    for l in trucks:
        model.addConstr(T[l] >= release_time * X[(i, j, k, l)])

In [111]:
# Constraint 3: Truck must arrive at the destination within the operational hours
for (i, j, k) in valid_combinations:
    start_shift = destination_df[destination_df['Destination_ID'] == j]['Start of shift'].values[0]
    end_shift = destination_df[destination_df['Destination_ID'] == j]['End of lay-on'].values[0]
    travel_time = trucking_df[(trucking_df['Origin_ID'] == i) & (trucking_df['Destination_ID'] == j)]['OSRM_time [sek]'].values[0] / 3600  # Convert to hours
    # print(f"Route {i}->{j}->{k} | Start Shift: {start_shift}, End Shift: {end_shift}, Travel Time: {travel_time}")
    for l in trucks:
        ArrivalTime[(l)] = (T[l] + travel_time * X[(i, j, k, l)] + 24 * quicksum((d-1)*ArrivalDayBinary[(l, d)] for d in range(1, 7))) - 24 * model.addVar(vtype=GRB.INTEGER, name=f"multiplier_{i}_{j}_{k}_{l}")
        model.addConstr(ArrivalTime[(l)] >= start_shift)
        model.addConstr(ArrivalTime[(l)] <= end_shift)

In [112]:
# 4. Each consignment must be assigned to exactly one truck
for (i, j, k) in valid_combinations:
    model.addConstr(quicksum(X[(i, j, k, l)]*Z[(l)] for l in trucks) == 1)

In [113]:
for j in destination_list:
    working_hours = destination_df[destination_df['Destination_ID'] == j]['End of lay-on'].values[0] - destination_df[destination_df['Destination_ID'] == j]['Start of shift'].values[0]
    sorting_capacity_per_day =  working_hours/2*destination_df[destination_df['Destination_ID'] == j]['Sorting capacity'].values[0]
    print()
    for d in range(1, 7):  # Maximum number of days to consider
        model.addConstr(
            quicksum(X[(i, j, k, l)] * source_df[source_df['id'] == k]['Consignment quantity'].values[0] * ArrivalDayBinary[(l, d)]
                     for i in source_list if j != i
                     for k in consignment_list
                     for l in trucks
                     if (i, j, k) in valid_combinations) <= sorting_capacity_per_day,
            name=f"SortingCapacity_{j}_{d}"
        )


In [114]:
for l in trucks:
    model.addConstr(
        quicksum(ArrivalDayBinary[(l, d)] * Z[l] for d in range(1, 7)) == 1,
        name = f'Assigning Arrival Day to each used truck'
        )

In [115]:
import time
class MyCallback:
    def __init__(self, model, improve_time_limit=5*60):
        self.model = model
        self.improve_time_limit = improve_time_limit
        self.last_improvement_time = time.time()
        self.best_obj_val = float('-inf')

    def __call__(self, model, where):
        if where == GRB.Callback.MIPSOL:
            current_obj_val = model.cbGet(GRB.Callback.MIPSOL_OBJ)
            if current_obj_val > self.best_obj_val:
                self.best_obj_val = current_obj_val
                self.last_improvement_time = time.time()
            elif time.time() - self.last_improvement_time > self.improve_time_limit:
                print(f"No improvement for {self.improve_time_limit / 60} minutes. Stopping optimization.")
                model.terminate()

In [116]:
model.setParam('TimeLimit', 5*60)

Set parameter TimeLimit to value 300


In [117]:
# Solve the model
callback = MyCallback(model)
model.optimize(callback)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1240P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 77700 rows, 54300 columns and 490500 nonzeros
Model fingerprint: 0x9ada05ef
Model has 1800 quadratic objective terms
Model has 392 quadratic constraints
Variable types: 600 continuous, 53700 integer (27900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  QMatrix range    [1e+00, 2e+03]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 1e+01]
  Bounds range     [1e+00, 2e+01]
  RHS range        [2e+01, 3e+01]
  QRHS range       [1e+00, 6e+04]
Presolve added 386 rows and 0 columns
Presolve removed 0 rows and 600 columns
Presolve time: 1.02s
Presolved: 232892 rows, 208500 columns, 1137000 nonzeros
Variable types: 300 continuous, 208200 integer (182400 binary)
Found heuristic solution: objective 496.

In [119]:
# Accessing ArrivalDay values after optimization
if model.status in [GRB.OPTIMAL, GRB.TIME_LIMIT, GRB.SUBOPTIMAL]:
    # Extract the data into a DataFrame
    data = []
    k_set = set()
    for (i, j, k, l) in X.keys():
        if (Z[l].X == 1 ) and (X[i,j,k,l].X == 1) :
            start_shift = destination_df[destination_df['Destination_ID'] == j]['Start of shift'].values[0]
            end_shift = destination_df[destination_df['Destination_ID'] == j]['End of lay-on'].values[0]
            travel_time = trucking_df[(trucking_df['Origin_ID'] == i) & (trucking_df['Destination_ID'] == j)]['OSRM_time [sek]'].values[0] / 3600
            arrival = quicksum(d*ArrivalDayBinary[(l, d)].X for d in range(1, 7))
            data.append({
                'Origin(PZA)': i,
                'Destination (PZE)': j,
                'Consignment ID': k,
                'Truck Id': l,
                'Departure time': T[l].X,
                'Arrival Day': arrival,
                "Destination Start Shift": start_shift,
                "Destination End Shift": end_shift,
                "Travel Time": travel_time
                
          })

In [120]:
df = pd.DataFrame(data)

In [ ]:
import os
path = os.getcwd()
df.to_csv("output/output.csv")

In [ ]:
model.write("file.lp")

In [121]:
df.head()

,Origin(PZA),Destination (PZE),Consignment ID,Truck Id,Departure time,Arrival Day,Destination Start Shift,Destination End Shift,Travel Time
0,04.1.1.PZ,01.1.1.PZ,7722434,142,22.0,1.0,22.0,30.416667,2.163972
1,04.1.1.PZ,01.1.1.PZ,7722435,123,22.0,1.0,22.0,30.416667,2.163972
2,04.1.1.PZ,01.1.1.PZ,7722436,291,22.0,2.0,22.0,30.416667,2.163972
3,04.1.1.PZ,01.1.1.PZ,7722437,285,22.0,2.0,22.0,30.416667,2.163972
4,04.1.1.PZ,01.1.1.PZ,7740142,293,22.0,2.0,22.0,30.416667,2.163972


In [122]:
df.tail()

,Origin(PZA),Destination (PZE),Consignment ID,Truck Id,Departure time,Arrival Day,Destination Start Shift,Destination End Shift,Travel Time
81,14.2.1.PZ,01.1.1.PZ,7935264,292,22.0,2.0,22.0,30.416667,2.971139
82,14.2.1.PZ,01.1.1.PZ,7935265,35,22.0,1.0,22.0,30.416667,2.971139
83,14.2.1.PZ,01.1.1.PZ,7935266,43,22.0,1.0,22.0,30.416667,2.971139
84,14.2.1.PZ,01.1.1.PZ,7937382,76,22.0,1.0,22.0,30.416667,2.971139
85,14.2.1.PZ,01.1.1.PZ,7937384,71,22.0,1.0,22.0,30.416667,2.971139
